<a href="https://colab.research.google.com/github/saad-the-noob/Skill_Morph_Assignment/blob/main/Assignment_5_Data_Preprocessing_and_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Load the Data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# TODO: Load the housing data
from google.colab import files       # from pc
uploaded = files.upload()
import io
df=pd.read_csv(io.BytesIO(uploaded['Housing(2).csv']))


Saving Housing(2).csv to Housing(2).csv


In [4]:
# TODO: How many houses are there?
print(f"Number of houses: ", len(df))  # Fill this

# TODO: Show first 3 houses
# Your code here
print(df.head(3))

Number of houses:  545
      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  


In [5]:
# Look at Columns
# TODO: Show all column names
print("Columns:")
# Your code here
print(df.columns.tolist())
# TODO: Show data types of columns
print("Data types:")
print(df.dtypes)

Columns:
['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'furnishingstatus']
Data types:
price                int64
area                 int64
bedrooms             int64
bathrooms            int64
stories              int64
mainroad            object
guestroom           object
basement            object
hotwaterheating     object
airconditioning     object
parking              int64
prefarea            object
furnishingstatus    object
dtype: object


In [17]:
#Convert Yes/No to 1/0
# Check what's in our data
# Shows: yes, no, yes, yes, no
# Models need numbers, not text!
# We need: yes → 1, no → 0
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for col in ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']:
 df[col] = label_encoder.fit_transform(df[col])

In [40]:
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus,mainroad_encoded,guestroom_encoded,basement_encoded,hotwaterheating_encoded,airconditioning_encoded,prefarea_encoded
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,0,1,0,0,0,1,1
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,0,1,0,0,0,1,0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1,1,0,1,0,0,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,0,1,0,1,0,1,1
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,0,1,1,1,0,1,0


In [34]:
# Split into Train and Test
# TODO: Split - 80% train, 20% test
x=df.drop(['price','mainroad_encoded','guestroom_encoded','basement_encoded',
           'hotwaterheating_encoded',"airconditioning_encoded","prefarea_encoded"],axis=1)
y=df['price']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=20)



In [35]:
#Apply StandardScaler
# Area is huge (1650-13300), Bedrooms is small (1-6)!
#Your Code here
scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)


In [36]:
#train and test model
#your code here
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(x_train_scaled, y_train)
accuracy = model.score(x_test_scaled, y_test)
train_score = model.score(x_train_scaled, y_train)
print(f"Training accuracy: {train_score:.3%}")
print(f"Accuracy: {accuracy:.3%}")

Training accuracy: 39.908%
Accuracy: 1.835%


In [37]:
import joblib

# Save the model
joblib.dump(model, 'my_model.pkl')
print("✅ Model saved as 'my_model.pkl'")

# Save the scaler
joblib.dump(scaler, 'my_scaler.pkl')
print("✅ Scaler saved as 'my_scaler.pkl'")

# Save scaled data as CSV
pd.DataFrame(x_train_scaled).to_csv('train_data_scaled.csv', index=False)
print("✅ Scaled data saved as 'train_data_scaled.csv'")


✅ Model saved as 'my_model.pkl'
✅ Scaler saved as 'my_scaler.pkl'
✅ Scaled data saved as 'train_data_scaled.csv'


In [41]:
# Predict a House Price
# New house details:
# area=5000, bedrooms=3, bathrooms=2, stories=2,
# mainroad=yes, guestroom=no, basement=yes,
# hotwaterheating=no, airconditioning=yes,
# parking=2, prefarea=yes, furnishingstatus=furnished
#your code here
model = joblib.load('my_model.pkl')
scaler = joblib.load('my_scaler.pkl')

new_house=pd.DataFrame([{
    "area": 5000, "bedrooms": 3, "bathrooms": 2, "stories": 2,
    "mainroad": 1, "guestroom": 0, "basement": 1,
    "hotwaterheating": 0, "airconditioning": 1,
    "parking": 2, "prefarea": 1,
    "furnishingstatus": 0
    }])
new_house_scaled = scaler.transform(new_house)
predicted_price = model.predict(new_house_scaled)
print(f"Predicted price: {predicted_price[0]}")

Predicted price: 6300000
